In [ ]:
import sys
sys.version

In [ ]:
# Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn


In [ ]:
tf.__version__

In [ ]:
batch_length = 500 # number of batches of time series
f_horizon = 250 # forecast horizon, n timesteps into the future
periods = 6000 + f_horizon

np.random.seed(111)

plt.rcParams['figure.figsize'] = [15, 7]

ext_val = -10*np.pi
rads = np.linspace(start=-ext_val, stop=ext_val, num=periods)

rads_rand = np.linspace(start=-5*ext_val, stop=5*ext_val, num=periods)
rads_rand = np.sin(rads_rand) * np.random.uniform(0.8, 1, size=periods)
rads_rand *= 0.8
plt.plot(rads_rand)
plt.show()

rand = np.random.uniform(-1, 1, size=periods).cumsum()
rand = rand / max(abs(rand)) #* 4
#rads = np.add(rads, rand)
#print(len(rads))
plt.plot(rand)
plt.show()
#ts = pd.Series(np.sin(rads) + rand + rads_rand, rads)#.cumsum()
#ts = pd.Series(np.sin(rads), rads)#.cumsum()

ts = pd.Series(rand)#.cumsum()


ts.plot(c='b', title='Example Time Series')
plt.show()
ts.head(10)

## Break data up into array that we can make batches of


In [ ]:
TS = np.array(ts)

ts_len = len(TS) - batch_length

# input data
x_data = TS[:(ts_len - ts_len % batch_length)]
print(len(x_data))
# get the time x values as bathces
x_batches = x_data.reshape(-1, batch_length, 1)

# do the same with the y values, but shift the values by how long into future we want to predict values
# y values are the true sequence coming after the x sequences
y_data = TS[f_horizon:(ts_len - (ts_len % batch_length)) + f_horizon]
print(len(y_data))
y_batches = y_data.reshape(-1, batch_length, 1)

print('y batches should be shifted by f_horizon values compared to x batches')

print('x_batches')
print(x_batches.shape)
#print(x_batches[0:2])

print('y_batches')
print(y_batches.shape)
#print(y_batches[0:2])





## Split data into training and testing set

In [ ]:
def split_data(x_batches, y_batches, train_proportion=0.8):
    num_batches = len(x_batches)
    num_train_batches = int(train_proportion * num_batches)
    
    X_train = x_batches[:num_train_batches]
    Y_train = y_batches[:num_train_batches]
    
    X_test = x_batches[num_train_batches:]
    Y_test = y_batches[num_train_batches:]
    
    return X_train, Y_train, X_test, Y_test

train_proportion = 0.8
X_train, Y_train, X_test, Y_test = split_data(x_batches, y_batches, train_proportion)

print(len(X_train))
print(len(X_test))

    

## Create tensorflow graph for computation

In [ ]:
tf.reset_default_graph() # Reset previous running graphs

inputs = 1
outputs = 1

X = tf.placeholder(tf.float32, [None, batch_length, inputs])
y = tf.placeholder(tf.float32, [None, batch_length, outputs])

hidden_units = 512

rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_units, activation=tf.nn.relu)
rnn_output, states = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)

learning_rate = 0.001

stacked_rnn_output = tf.reshape(rnn_output, [-1, hidden_units])
stacked_outputs = tf.layers.dense(stacked_rnn_output, outputs)
out = tf.reshape(stacked_outputs, [-1, batch_length, outputs])

loss = tf.reduce_sum(tf.square(out - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)


## Training time 

In [ ]:
epochs = 1000

train_graph = tf.Graph()
with train_graph.as_default():
    global_step = tf.train.create_global_step()

#with tf.Session() as sess:
    with tf.train.MonitoredSession() as sess:
        while not sess.should_stop():

        #sess.run(tf.global_variables_initializer())
        #for ep in range(epochs):
            sess.run(training_op, feed_dict={X:X_train, y:Y_train})

        if ep % 100 == 0:
            mse = sess.run(loss, feed_dict={X:X_test, y:Y_test})
            print(ep, '\tMSE:', mse)
    y_pred_train = sess.run(out, feed_dict={X:X_train})
    y_pred_test = sess.run(out, feed_dict={X:X_test})


## Plotting time

In [ ]:
plt.title('Forecast vs actual', fontsize=14)
#plt.plot(pd.Series(np.ravel(Y_train)), 'y', markersize=10, label='actual')
#plt.plot(pd.Series(np.ravel(y_pred_train)), 'g', markersize=10, label='actual')

plt.plot(pd.Series(np.ravel(Y_test)), 'b', markersize=10, label='actual')
plt.plot(pd.Series(np.ravel(y_pred_test)), 'r', markersize=10, label='forecast')


xcoords = np.array([x for x in range(0, len(np.ravel(Y_test)), batch_length)])
xcoords2 = xcoords + batch_length - f_horizon

for xc in xcoords:
    plt.axvline(x=xc, color='k')

for xc in xcoords2:
    plt.axvline(x=xc, color='g')


plt.legend(loc='upper left')
plt.xlabel('time periods')

plt.show()